# Advanced Level

# Task 8 : Next Word Prediction

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize
import re
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Activation

In [20]:
file='/kaggle/input/1661-0-txt/1661-0.txt'

In [21]:
text=open(file).read().lower()

In [22]:
print('Length of Corpus is: ',len(text))

Length of Corpus is:  581888


In [23]:
words=re.findall(r'[a-zA-Z]+',text)

In [24]:
filtered_text=" ".join(re.findall(r'[a-zA-Z]+',text))

In [25]:
words=word_tokenize(filtered_text)

In [26]:
len(words)

109076

In [27]:
unique_words=np.unique(words)

In [28]:
len(np.unique(words))

8081

In [29]:
unique_words_dict=dict((j,i) for i, j in enumerate(unique_words))

In [30]:
number_words=5
prev_words=[]
next_word=[]
for i in range(len(words)-number_words):
    prev_words.append(words[i:i+number_words])
    next_word.append(words[i+number_words])

In [31]:
print(prev_words[4])
print(next_word[4])

['adventures', 'of', 'sherlock', 'holmes', 'by']
arthur


In [32]:
X=np.zeros((len(prev_words),number_words,len(unique_words)),dtype='bool')
Y=np.zeros((len(next_word),len(unique_words)),dtype='bool')

In [33]:
X.shape

(109071, 5, 8081)

In [34]:
Y.shape

(109071, 8081)

In [35]:
for i, every_words in enumerate(prev_words):
    for j , every_word in enumerate(every_words):
        X[i,j,unique_words_dict[every_word]]=1
    Y[i,unique_words_dict[next_word[i]]]=1

In [36]:
X[1].shape

(5, 8081)

In [37]:
np.where(X[1]==True)

(array([0, 1, 2, 3, 4]), array([3273, 6039, 7139,  132, 4844]))

In [38]:
model=Sequential()

In [39]:
model.add(GRU(64,return_sequences=True,input_shape=(number_words,len(unique_words))))
model.add(GRU(64))
model.add(Dropout(0.2))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 5, 64)             1564224   
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 8081)              525265    
                                                                 
 activation_1 (Activation)   (None, 8081)              0         
                                                                 
Total params: 2,114,449
Trainable params: 2,114,449
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [42]:
history=model.fit(X,Y,validation_split=0.05,batch_size=128,epochs=30,shuffle=True)

Epoch 1/30
810/810 [==============================] - 83s 97ms/step - loss: 6.6884 - accuracy: 0.0518 - val_loss: 7.5676 - val_accuracy: 0.0550
Epoch 2/30
810/810 [==============================] - 77s 95ms/step - loss: 6.4753 - accuracy: 0.0532 - val_loss: 7.5909 - val_accuracy: 0.0550
Epoch 3/30
810/810 [==============================] - 77s 96ms/step - loss: 6.3186 - accuracy: 0.0537 - val_loss: 7.5593 - val_accuracy: 0.0644
Epoch 4/30
810/810 [==============================] - 76s 94ms/step - loss: 6.0719 - accuracy: 0.0679 - val_loss: 7.4639 - val_accuracy: 0.0732
Epoch 5/30
810/810 [==============================] - 76s 93ms/step - loss: 5.8415 - accuracy: 0.0865 - val_loss: 7.3895 - val_accuracy: 0.0836
Epoch 6/30
810/810 [==============================] - 75s 93ms/step - loss: 5.6152 - accuracy: 0.1060 - val_loss: 7.4006 - val_accuracy: 0.0875
Epoch 7/30
810/810 [==============================] - 76s 94ms/step - loss: 5.4341 - accuracy: 0.1208 - val_loss: 7.5180 - val_accuracy:

In [43]:
def predict_next_word(input,best_word=5):
    input=input.lower()
    X=np.zeros((1,number_words,len(unique_words)))
    for i, word in enumerate(input.split()):
        X[0,i,unique_words_dict[word]]=1
    prediction=model.predict(X)
    return np.argpartition(prediction,-best_word)

In [44]:
possible_words=predict_next_word('for the use of anyone')[0][:5]

1/1 [==============================] - 1s 809ms/step


In [45]:
[unique_words[i] for i in possible_words]

['greatest', 'zip', 'abandons', 'abbots', 'aberdeen']

In [46]:
prev_words[15]
next_word[15]

'anywhere'